# Superpixel Segmentation
### 画像内を幾つかの色や距離が似た集団毎で小領域に分割する手法

- https://sabopy.com/py/scikit-image-62/#toc1
- https://namonakihito.hatenablog.com/entry/2018/08/28/155306
- https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_segmentations.html
- http://sabaku2017.hatenablog.com/entry/2018/01/28/170525

---

### ライブラリソース修正
### 　~/.local/lib/python3.6/site-packages/skimage/future/graph/rag.py
### 　 graph.node[n] ===> graph.node<font color="HotPink">**s**</font>[n]

---

In [1]:
import os, sys, pprint
import cv2
import skimage
from skimage import data, io, segmentation, color
from skimage.future import graph
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
%matplotlib inline

In [2]:
print(skimage.__version__) # 0.15.0

0.15.0


### 
### 1. オリジナル画像の読み込み

In [3]:
#img_file = './image/dish02.jpg'
img_file = './image/dish02.png'
ext_type = os.path.splitext(img_file)[1]
print('ext_type : {}'.format(ext_type))
tmp = cv2.imread(img_file, cv2.IMREAD_UNCHANGED)

if ext_type == '.jpg':
    wk_img = np.copy(tmp)
    print(wk_img.shape)

elif ext_type == '.png':
    # PNGの場合
    # 3次元目の要素(チャネル)の数を４->3へ
    # チャネル数にアルファ(透過)が含まれているので削除
    wk_img = np.delete(tmp, obj=3, axis=2)
    print(wk_img.shape)

# OpenCV -> matplotlib
img = cv2.cvtColor(wk_img, cv2.COLOR_BGR2RGB) 

plt.figure(figsize=(10, 5))
plt.title('original', size=20)
plt.axis('off')
plt.imshow(img)
pass # dummy

ext_type : .png


NameError: name 'tmp' is not defined

### 
### 2. SLIC(Simple Linear Iterative Clustering)
- 画像をsuperpixelに分割する代表的なアルゴリズム
- K-Means法をベースにしている
- superpixel ： 距離的、色的に近い画素をひとまとまりにした領域
- https://scikit-image.org/docs/0.15.x/api/skimage.segmentation.html#module-skimage.segmentation

In [ ]:
# セグメンテーション毎にラベル付け
# compactness : Balances color proximity and space proximity. (値を大きくするとsuperpixelが正方形に近づく)
# n_segments : The (approximate) number of labels in the segmented output image.
labels = segmentation.slic(img, compactness=30, n_segments=400)
print(type(labels))
print(labels.shape)

# ラベル付けされた部分への色付け
# kind : 'avg’ replaces each labeled segment with its average color.
slic_img = color.label2rgb(labels, img, kind='avg')

# ラベル間の境界線を描く
# color : RGB color of boundaries in the output image. e.g.(0.5, 0.5, 0)
slic_img_bound = segmentation.mark_boundaries(slic_img, labels, (0, 0, 0))

In [ ]:
# matplotlib表示
fig = plt.figure(figsize=(25,10), tight_layout=True)
ax0 = plt.subplot2grid((2,3), (0,0), rowspan=1, colspan=1)
ax1 = plt.subplot2grid((2,3), (1,0), rowspan=1, colspan=1)
ax2 = plt.subplot2grid((2,3), (1,1), rowspan=1, colspan=1)
ax3 = plt.subplot2grid((2,3), (1,2), rowspan=1, colspan=1)

ax0.set_title('original', size=20)
ax1.set_title('slic - labels', size=20)
ax2.set_title('label2rgb', size=20)
ax3.set_title('mark_boundaries', size=20)
ax0.axis('off')
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')
ax0.imshow(img)
ax1.imshow(labels)
ax2.imshow(slic_img)
ax3.imshow(slic_img_bound)

pass

### 
### 3. 領域隣接グラフ (Region Adjacency Graph)
- https://scikit-image.org/docs/dev/api/skimage.future.graph.html#skimage.future.graph.merge_hierarchical
- https://sabopy.com/py/scikit-image-62/#toc1

In [ ]:
# 「オリジナル画像」と「セグメンテーション毎のラベル情報」から、領域隣接グラフ(Region Adjacency Graph)を構築する
rag = graph.rag_mean_color(img, labels)
print(type(rag))

In [ ]:
# 結合する領域の平均色を求める関数
# https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_rag_merge.html#sphx-glr-auto-examples-segmentation-plot-rag-merge-py
def merge_mean_color(graph, src, dst):
    """Callback called before merging two nodes of a mean color distance graph.
    This method computes the mean color of `dst`.
    Parameters
    ----------
    graph : RAG
        The graph under consideration.
    src, dst : int
        The vertices in `graph` to be merged.
    """
    graph.nodes[dst]['total color'] += graph.nodes[src]['total color']
    graph.nodes[dst]['pixel count'] += graph.nodes[src]['pixel count']
    graph.nodes[dst]['mean color'] = (graph.nodes[dst]['total color'] /
                                     graph.nodes[dst]['pixel count'])

In [ ]:
# 平均色を再計算してノードの差を求める関数
def _weight_mean_color(graph, src, dst, n):
    """Callback to handle merging nodes by recomputing mean color.
    The method expects that the mean color of `dst` is already computed.
    Parameters
    ----------
    graph : RAG
        The graph under consideration.
    src, dst : int
        The vertices in `graph` to be merged. #vertices:頂点
    n : int
        A neighbor of `src` or `dst` or both.
    Returns
    -------
    data : dict
        A dictionary with the `"weight"` attribute set as the absolute
        difference of the mean color between node `dst` and `n`.
    """

    diff = graph.nodes[dst]['mean color'] - graph.nodes[n]['mean color']
    diff = np.linalg.norm(diff)
    return {'weight': diff}

In [ ]:
# Perform hierarchical merging of a RAG
# セグメンテーション毎にラベル付け
labels_rag = graph.merge_hierarchical(
                                    labels,
                                    rag,
                                    thresh=35,
                                    rag_copy=False,
                                    in_place_merge=True,
                                    merge_func=merge_mean_color,
                                    weight_func=_weight_mean_color
                                  )

# ラベル付けされた部分への色付け
# kind : 'avg’ replaces each labeled segment with its average color.
rag_img = color.label2rgb(labels_rag, img, kind='avg')

# ラベル間の境界線を描く
# color : RGB color of boundaries in the output image. e.g.(0.5, 0.5, 0)
rag_img_bound = segmentation.mark_boundaries(rag_img, labels_rag, (0, 0, 0))

In [ ]:
# matplotlib表示
fig = plt.figure(figsize=(25,10), tight_layout=True)
ax0 = plt.subplot2grid((2,3), (0,0), rowspan=1, colspan=1)
ax1 = plt.subplot2grid((2,3), (1,0), rowspan=1, colspan=1)
ax2 = plt.subplot2grid((2,3), (1,1), rowspan=1, colspan=1)
ax3 = plt.subplot2grid((2,3), (1,2), rowspan=1, colspan=1)

ax0.set_title('original', size=20)
ax1.set_title('RAG - labels', size=20)
ax2.set_title('label2rgb', size=20)
ax3.set_title('mark_boundaries', size=20)
ax0.axis('off')
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')
ax0.imshow(img)
ax1.imshow(labels_rag)
ax2.imshow(rag_img)
ax3.imshow(rag_img_bound)

pass

### 
### 4. 領域隣接グラフ (Region Adjacency Graph)　※閾値変更(35→50)

In [ ]:
# Perform hierarchical merging of a RAG
# セグメンテーション毎にラベル付け
labels_rag50= graph.merge_hierarchical(
                                    labels,
                                    rag,
                                    thresh=50, # 35->50へ変更
                                    rag_copy=False,
                                    in_place_merge=True,
                                    merge_func=merge_mean_color,
                                    weight_func=_weight_mean_color
                                  )

# ラベル付けされた部分への色付け
# kind : 'avg’ replaces each labeled segment with its average color.
rag_img50 = color.label2rgb(labels_rag50, img, kind='avg')

# ラベル間の境界線を描く
# color : RGB color of boundaries in the output image. e.g.(0.5, 0.5, 0)
rag_img_bound50 = segmentation.mark_boundaries(rag_img50, labels_rag50, (0, 0, 0))

In [ ]:
# matplotlib表示
fig = plt.figure(figsize=(25,10), tight_layout=True)
ax0 = plt.subplot2grid((2,3), (0,0), rowspan=1, colspan=1)
ax1 = plt.subplot2grid((2,3), (1,0), rowspan=1, colspan=1)
ax2 = plt.subplot2grid((2,3), (1,1), rowspan=1, colspan=1)
ax3 = plt.subplot2grid((2,3), (1,2), rowspan=1, colspan=1)

ax0.set_title('original', size=20)
ax1.set_title('RAG - labels', size=20)
ax2.set_title('label2rgb', size=20)
ax3.set_title('mark_boundaries', size=20)
ax0.axis('off')
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')
ax0.imshow(img)
ax1.imshow(labels_rag50)
ax2.imshow(rag_img50)
ax3.imshow(rag_img_bound50)

pass

### 
### 5. まとめ

In [ ]:
# matplotlib表示
fig = plt.figure(figsize=(25,15), tight_layout=True)
ax0 = plt.subplot2grid((3,3), (0,0), rowspan=1, colspan=1)
ax1 = plt.subplot2grid((3,3), (1,0), rowspan=1, colspan=1)
ax2 = plt.subplot2grid((3,3), (1,1), rowspan=1, colspan=1)
ax3 = plt.subplot2grid((3,3), (1,2), rowspan=1, colspan=1)
ax4 = plt.subplot2grid((3,3), (2,0), rowspan=1, colspan=1)
ax5 = plt.subplot2grid((3,3), (2,1), rowspan=1, colspan=1)
ax6 = plt.subplot2grid((3,3), (2,2), rowspan=1, colspan=1)

ax0.set_title('original', size=20)
ax1.set_title('slic', size=20)
ax2.set_title('RAG(thresh:35)', size=20)
ax3.set_title('RAG(thresh:50)', size=20)
ax4.set_title('slic - mark_boundaries', size=20)
ax5.set_title('RAG(thresh:35) - mark_boundaries', size=20)
ax6.set_title('RAG(thresh:50) - mark_boundaries', size=20)

ax0.axis('off')
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')
ax4.axis('off')
ax5.axis('off')
ax6.axis('off')

ax0.imshow(img)
ax1.imshow(slic_img)
ax2.imshow(rag_img)
ax3.imshow(rag_img50)
ax4.imshow(slic_img_bound)
ax5.imshow(rag_img_bound)
ax6.imshow(rag_img_bound50)

pass